<a href="https://colab.research.google.com/github/SarunDz-12/229352-Statistical-Learning-for-Data-Science-2/blob/main/Lab02_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #2

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve

# For Fashion-MNIST
from tensorflow.keras.datasets import fashion_mnist

# For 20 Newsgroups
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## Part 1: Marketing Campaign Dataset - Manual Data Preprocessing & Logistic Regression

### Load the Marketing Campaign Dataset ([Data Information](https://archive.ics.uci.edu/dataset/222/bank+marketing))

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be (`'yes'`) or not (`'no'`) subscribed.

In [5]:
bank_url = 'https://raw.githubusercontent.com/donlap/ds352-labs/main/bank.csv'

df = pd.read_csv(bank_url, sep=';', na_values=['unknown'])
df = df.drop(["emp.var.rate", "cons.price.idx", "cons.conf.idx",	"euribor3m", "nr.employed"], axis=1)
print("Shape of the dataset:", df.shape)
df.head()

Shape of the dataset: (41188, 16)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,no
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,149,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,no


### Data Exploration

In [6]:
print("--- Missing Values Count ---")
print(df.isnull().sum())

--- Missing Values Count ---
age               0
job             330
marital          80
education      1731
default        8597
housing         990
loan            990
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
y                 0
dtype: int64


In [7]:
print("--- Unique Values for Categorical Columns ---")
for col in df.select_dtypes(include='object').columns:
    print(f"\n'{col}' unique values:")
    print(df[col].value_counts(dropna=False)) # Include NaN counts

--- Unique Values for Categorical Columns ---

'job' unique values:
job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
NaN                330
Name: count, dtype: int64

'marital' unique values:
marital
married     24928
single      11568
divorced     4612
NaN            80
Name: count, dtype: int64

'education' unique values:
education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
NaN                     1731
illiterate                18
Name: count, dtype: int64

'default' unique values:
default
no     32588
NaN     8597
yes        3
Name: count, dtype: int64

'housing' unique values:
housing
yes    21576
no     18622
NaN      990
Name: count, dtype: int64


### Data Preprocessing

In [8]:
# Map target variable 'y' to 0 (no) and 1 (yes)
df['y_new'] = df['y'].map({'yees':1, 'no':0}) # Write your code here


# Drop 'duration' due to data leakage


# Define features (X) and target (y)

y = df['y_new']
x = df.drop(['y', 'y_new'], axis=1)
# Split the data BEFORE any transformations


# Print data shape
print(y)
print(x)


0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
41183    NaN
41184    0.0
41185    0.0
41186    NaN
41187    0.0
Name: y_new, Length: 41188, dtype: float64
       age          job  marital            education default housing loan    contact month day_of_week  duration  campaign  pdays  previous     poutcome
0       56    housemaid  married             basic.4y      no      no   no  telephone   may         mon       261         1    999         0  nonexistent
1       57     services  married          high.school     NaN      no   no  telephone   may         mon       149         1    999         0  nonexistent
2       37     services  married          high.school      no     yes   no  telephone   may         mon       226         1    999         0  nonexistent
3       40       admin.  married             basic.6y      no      no   no  telephone   may         mon       151         1    999         0  nonexistent
4       56     services  married          hi

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3)

x_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
16658,30,technician,divorced,professional.course,no,no,yes,cellular,jul,wed,808,1,999,0,nonexistent
27651,40,admin.,single,university.degree,no,yes,yes,cellular,nov,fri,12,10,999,0,nonexistent
28276,41,blue-collar,married,basic.4y,no,yes,no,cellular,apr,wed,123,1,999,0,nonexistent
14230,35,admin.,single,university.degree,no,yes,no,cellular,jul,mon,77,2,999,0,nonexistent
40438,56,blue-collar,divorced,basic.4y,no,yes,no,telephone,aug,mon,361,1,12,4,failure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24126,56,services,married,high.school,no,no,no,telephone,nov,mon,92,1,999,0,nonexistent
36079,34,housemaid,married,university.degree,no,yes,no,cellular,may,tue,58,1,999,0,nonexistent
9134,48,unemployed,married,high.school,NaN,yes,no,telephone,jun,fri,334,1,999,0,nonexistent
18745,43,admin.,married,professional.course,NaN,yes,no,cellular,jul,thu,747,4,999,0,nonexistent


We will apply `StandardScaler()`, `OrdinalEncoder()`, and `OneHotEncoder()` on a few selected columns.

**1. Numerical Feature: `age` and `campaign` (Standard Scaling)**

In [10]:
num_cols_demo = ['age', 'campaign']

scaler = StandardScaler()

# Fit the scaler ONLY on the training data


x_train[num_cols_demo] = scaler.fit_transform(x_train[num_cols_demo]) # Write your code here
x_test[num_cols_demo] = scaler.transform(x_test[num_cols_demo]) # Write your code here

x_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
16658,-0.962939,technician,divorced,professional.course,no,no,yes,cellular,jul,wed,808,-0.562678,999,0,nonexistent
27651,-0.003010,admin.,single,university.degree,no,yes,yes,cellular,nov,fri,12,2.653035,999,0,nonexistent
28276,0.092983,blue-collar,married,basic.4y,no,yes,no,cellular,apr,wed,123,-0.562678,999,0,nonexistent
14230,-0.482975,admin.,single,university.degree,no,yes,no,cellular,jul,mon,77,-0.205376,999,0,nonexistent
40438,1.532877,blue-collar,divorced,basic.4y,no,yes,no,telephone,aug,mon,361,-0.562678,12,4,failure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24126,1.532877,services,married,high.school,no,no,no,telephone,nov,mon,92,-0.562678,999,0,nonexistent
36079,-0.578968,housemaid,married,university.degree,no,yes,no,cellular,may,tue,58,-0.562678,999,0,nonexistent
9134,0.764934,unemployed,married,high.school,NaN,yes,no,telephone,jun,fri,334,-0.562678,999,0,nonexistent
18745,0.284969,admin.,married,professional.course,NaN,yes,no,cellular,jul,thu,747,0.509227,999,0,nonexistent


In [11]:
x_train.describe()

,age,duration,campaign,pdays,previous
count,2.883100e+04,28831.000000,2.883100e+04,28831.000000,28831.000000
mean,-3.105282e-17,258.920918,-4.140376e-17,961.630190,0.173390
std,1.000017e+00,260.741001,1.000017e+00,188.980496,0.495915
min,-2.210848e+00,0.000000,-5.626776e-01,0.000000,0.000000
25%,-7.709534e-01,102.000000,-5.626776e-01,999.000000,0.000000
50%,-1.949958e-01,179.000000,-2.053761e-01,999.000000,0.000000
75%,6.689407e-01,320.000000,1.519253e-01,999.000000,0.000000
max,5.564581e+00,3509.000000,1.908890e+01,999.000000,7.000000


Let's take a look at the transformed `age` and `campaign` features and their statistics.

In [12]:
print("\nOriginal X_train 'age' and 'campaign' head:")
print(x_train[num_cols_demo].head())
print("\nScaled X_train 'age' and 'campaign' head:")
print(pd.DataFrame(X_train_scaled_demo, columns=num_cols_demo, index=x_train.index).head())

print("\nMean of scaled 'age' (train):", X_train_scaled_demo[:, 0].mean())
print("Std Dev of scaled 'campaign' (train):", X_train_scaled_demo[:, 1].std())


Original X_train 'age' and 'campaign' head:
            age  campaign
6851  -0.482975  0.151925
25728  0.476955 -0.562678
34132 -0.866946  0.866528
24350 -0.290989 -0.205376
8460   0.092983  1.223830

Scaled X_train 'age' and 'campaign' head:


NameError: name 'X_train_scaled_demo' is not defined

**2. Ordinal Feature: `education` (Ordinal Encoding with Imputation)**

- **Imputation**

In [13]:
ord_col_demo = ['education']

imputer_ord = SimpleImputer(strategy='most_frequent')

## Write your code here

X_train[ord_col_demo] = imputer_ord.fit_transform(X_train[ord_col_demo]) # Write your code here
X_test[ord_col_demo] = imputer_ord.transform(X_test[ord_col_demo])  # Write your code here



NameError: name 'X_train' is not defined

- **Ordinal Encoding**

In [ ]:
education_categories = [
    'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
    'professional.course', 'university.degree', 'masters', 'doctorate'
]

In [14]:
ordinal_encoder = OrdinalEncoder(categories=[education_categories])

## Write your code here

x_train[ord_col_demo] = ordinal_encoder.fit_transform(x_train[ord_col_demo])
x_test[ord_col_demo] = ordinal_encoder.transform(x_test[ord_col_demo])

x_test

NameError: name 'education_categories' is not defined

Let's take a look at the imputed and ordinal-encoded `education`.

In [ ]:
print("\nOriginal X_train 'education' head:")
print(X_train[ord_col_demo].iloc[20:25])
print("\nImputed X_train 'education' head (after imputer.transform):")
print(pd.DataFrame(X_train_imputed_ord_demo, columns=ord_col_demo, index=X_train.index).iloc[20:25])
print("\nOrdinal Encoded X_train 'education' head:")
print(pd.DataFrame(X_train_ord_encoded_demo, columns=ord_col_demo, index=X_train.index).iloc[20:25])

**3. Nominal Feature: `job` (One-Hot Encoding with Imputation)**

- **Imputation**

In [15]:
nom_col_demo = ['job']

imputer_nom = SimpleImputer(strategy='most_frequent')
imputer_nom.fit(X_train[nom_col_demo])

X_train[nom_col_demo] = imputer_nom.fit_transform(X_train[nom_col_demo])
X_test[nom_col_demo] = imputer_nom.transform(X_test[nom_col_demo])

NameError: name 'X_train' is not defined

- **Nominal Encoding**

In [16]:
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

## Write your code here


X_train_onehot = onehot_encoder.fit_transform(x_train[['job']]) ## Write your code here
X_test_onehot = onehot_encoder.transform(x_test[['job']]) ## Write your code here

X_train_onehot.shape

(28831, 12)

In [17]:
x_train = pd.concat([x_train.reset_index(drop=True),
 pd.DataFrame(x_train_onehot, columns=onehot_encoder.get_feature_names_out(['job']))] axis = 1)

x_train

SyntaxError: invalid syntax. Perhaps you forgot a comma? (ipython-input-2550897293.py, line 1)

In [18]:
x_test_test = pd.concat([x_test.reset_index(drop=True),
                        pd.DataFrame(x_test_onehot, columns=onehot_encoder.get_feature_names_out(['job']))] axis = 1)

x_test

SyntaxError: invalid syntax. Perhaps you forgot a comma? (ipython-input-3738230996.py, line 1)

In [19]:
print("\nOriginal X_train 'job' head:")
print(X_train[nom_col_demo].iloc[40:45])
print("\nImputed X_train 'job' head (after imputer.transform):")
print(pd.DataFrame(X_train_imputed_nom_demo, columns=nom_col_demo, index=X_train.index).iloc[40:45])
print("\nOne-Hot Encoded X_train 'job' shape:", X_train_onehot_encoded_demo.shape)
print("First 5 rows of One-Hot Encoded X_train 'job':")
print(pd.DataFrame(X_train_onehot_encoded_demo, columns=onehot_encoder.get_feature_names_out(nom_col_demo), index=X_train.index).iloc[40:45])


Original X_train 'job' head:


NameError: name 'X_train' is not defined

### **Exercise 1: Apply All Preprocessing & Train Logistic Regression**

Now, it's your turn to apply these preprocessing steps to *all* relevant columns and then train a Logistic Regression model.

**Instructions:**

1.  Look at the Variable Table in [this link](https://archive.ics.uci.edu/dataset/222/bank+marketing).
2. Make lists for `numerical_features`, `ordinal_features`, and `nominal_features`.
3. Preprocess the features. It is safer to make a copy of `X_train` using:
   ```
   X_train_copy = X_train.copy()
   X_test_copy = X_test.copy()
   ```
   and preprocess `X_train_copy` instead.

   **For nominal features, concat the one-hot encoded features using [`pd.concat(..., axis=1)`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) and drop the old nominal features from the dataframe.**
4. Train Logistic Regression on the preprocessed `X_train_copy` and `y_train`.
5. Evaluate the Model:
    *   Make predictions on the preprocessed `X_test_copy`.
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?


In [20]:
# --- YOUR CODE FOR EXERCISE 1 STARTS HERE ---
import pandas as pd
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

numerical_features = ['num1', 'num2', 'num3']
ordinal_features = ['ord1', 'ord2']
nominal_features = ['cat1', 'cat2']

In [21]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y', 'y_new'], dtype='object')

In [26]:
import pandas as pd

X_train_copy = X_train.copy()
X_test_copy = X_test.copy()



In [27]:
nominal_cols = ['cat_col1', 'cat_col2']  # replace with your real column names

In [33]:
categorical_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']
for col in categorical_cols:
    X_train_copy[col] = X_train_copy[col].astype('category')
    X_test_copy[col] = X_test_copy[col].astype('category')

In [31]:
print(X_train_copy.columns.tolist())

['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']


In [34]:
X_train_dummies = pd.get_dummies(X_train_copy[categorical_cols], drop_first=True)
X_test_dummies = pd.get_dummies(X_test_copy[categorical_cols], drop_first=True)
X_test_dummies = X_test_dummies.reindex(columns=X_train_dummies.columns, fill_value=0)

X_train_copy = pd.concat([X_train_copy.drop(columns=categorical_cols), X_train_dummies], axis=1)
X_test_copy = pd.concat([X_test_copy.drop(columns=categorical_cols), X_test_dummies], axis=1)

In [37]:
# ถ้า y_train ยังเป็น 'yes', 'no'
y_train_numeric = y_train.map({'yes': 1, 'no': 0})

In [38]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train_copy, y_train_numeric)

ValueError: could not convert string to float: 'no'

In [36]:
print(X_train_copy.dtypes)

age                               int64
duration                          int64
campaign                          int64
pdays                             int64
previous                          int64
y                                object
job_blue-collar                    bool
job_entrepreneur                   bool
job_housemaid                      bool
job_management                     bool
job_retired                        bool
job_self-employed                  bool
job_services                       bool
job_student                        bool
job_technician                     bool
job_unemployed                     bool
marital_married                    bool
marital_single                     bool
education_basic.6y                 bool
education_basic.9y                 bool
education_high.school              bool
education_illiterate               bool
education_professional.course      bool
education_university.degree        bool
default_yes                        bool


In [39]:
from sklearn.metrics import classification_report

# 1. แปลง y_test เป็นเลขให้เหมือนกับ y_train ที่ใช้เทรน
y_test_numeric = y_test.map({'yes': 1, 'no': 0})

# 2. ทำ Prediction บน X_test_copy
y_pred = lr_model.predict(X_test_copy)

# 3. Print classification_report
report = classification_report(y_test_numeric, y_pred, output_dict=True)
print(classification_report(y_test_numeric, y_pred))

# 4. Extract metrics ที่ต้องการ
accuracy = report['accuracy']
avg_precision = report['weighted avg']['precision']
avg_recall = report['weighted avg']['recall']
avg_f1 = report['weighted avg']['f1-score']

print(f"\nAccuracy: {accuracy:.4f}")
print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall: {avg_recall:.4f}")
print(f"Average F1-score: {avg_f1:.4f}")

ValueError: could not convert string to float: 'no'

## Part 2: Fashion-MNIST Dataset - Image Classification

### Load Fashion-MNIST Dataset

The Fashion-MNIST dataset consists of 28x28 grayscale images of fashion items.

In [ ]:
(fm_X_train, fm_y_train), (fm_X_test, fm_y_test) = fashion_mnist.load_data()

print(f"Fashion-MNIST Train data shape: {fm_X_train.shape}")
print(f"Fashion-MNIST Train labels shape: {fm_y_train.shape}")
print(f"Fashion-MNIST Test data shape: {fm_X_test.shape}")
print(f"Fashion-MNIST Test labels shape: {fm_y_test.shape}")

In [ ]:
print(f"First image {fm_X_train[0]}")
print(f"First label {fm_y_train[0]}")

### Visualize Fashion-MNIST Images

Let's see what these images look like.

In [ ]:
fashion_mnist_class_names = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

# Visualize the images
## Write your code here



### **Exercise 2: Preprocessing Images (Flatten and Scale)**

Images are 2D arrays (matrices of pixels) and pixel values are integers from 0-255. For Logistic Regression, we need:
*  **Flattening:** Convert each 28x28 image into a 1D array of 784 features.
*  **Scaling:** Normalize pixel values from [0, 255] to [0, 1].

**Instructions:**

1.   **Flatten:** Use the `.reshape()` method (see [documentation](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.reshape.html)). For `fm_X_train_binary` (shape `(num_samples, 28, 28)`), you want to reshape it to `(num_samples, 28*28)`.
2.  **Scale:** Divide the flattened pixel values by 255.0 to get values between 0 and 1.
3.   **Train Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
    *   Fit the model on your *processed* `fm_X_train_scaled` and `fm_y_train`.
4.   **Make Predictions:** Use `predict()` to make predictions on the *processed* `fm_X_test_scaled`.
5.   **Print Classification Report:** Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?
6.   **Visualize Misclassifications:**
    *   Find the indices in `fm_X_test_binary` where your model made incorrect predictions (i.e., `fm_y_pred != fm_y_test`).
    *   Select 5 of these misclassified images.
    *   Plot these images (using `plt.imshow`). For each image, print its true label and its predicted label.

In [ ]:
# --- YOUR CODE FOR EXERCISE 2 STARTS HERE ---





## Part 3: 20 Newsgroups Dataset - Text Classification

### Load 20 Newsgroups Dataset

The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics.

In [ ]:
news_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
news_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)

X_train_news, y_train_news = news_train.data, news_train.target
X_test_news, y_test_news = news_test.data, news_test.target

print(f"Number of training documents: {len(X_train_news)}")
print(f"Number of test documents: {len(X_test_news)}")
print(f"Categories: {news_train.target_names}")

### Explore Sample Document

In [ ]:
# Print the first document and its class
## Write your code here



### Preprocessing: Text Vectorization Demonstration with `TfidfVectorizer`

$$
\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)
$$

Where:

$$
\text{TF}(t, d) = \frac{\text{number of word }t\text{ in } d}{\text{number of words in } d} \quad \text{ and } \quad
\text{IDF}(t, D) = \log\left(\frac{\text{total number of documents}}{\text{number of documents that contain word }t}\right).
$$

In [ ]:
sample_sentences = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?"
]

vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the sample sentences
sample_vec_output_sparse = # Write your code here

sample_vec_output_dense = sample_vec_output_sparse.toarray()

print(vectorizer.vocabulary_)
print(vectorizer.get_feature_names_out())
print(sample_vec_output_dense)

### **Exercise 3: Apply TF-IDF Vectorization to Full Dataset**

Now, apply `TfidfVectorizer` to the actual training and testing datasets for the 20 Newsgroups classification task.

**Instructions:**

1.  **Initialize `TfidfVectorizer`:**
    *   Initialize `TfidfVectorizer`. Use `stop_words='english'` to remove common words.
2.  **Fit and Transform Training Data:**
    *   Call `fit_transform()` on `X_train_news` to learn the vocabulary and transform the training text into TF-IDF features. Store the result in `X_train_vec`.
3.  **Transform Test Data:**
    *   Call `transform()` on `X_test_news` using the *already fitted* vectorizer. Store the result in `X_test_vec`. **Crucially, do not call `fit_transform()` on the test data!** This would cause data leakage.
4.  **Initialize Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
5.  **Train the Model:**
    *   Fit the model on your `X_train_vec` and `y_train_news`.
6.  **Make Predictions:**
    *   Make predictions using `predict()` on the `X_test_vec`.
7.  **Evaluate the Model:**
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?

In [ ]:
# --- YOUR CODE FOR EXERCISE 3 STARTS HERE ---

num_cols_demo = ['age', 'campaign']

scaler = StandardScaler()

# Fit the scaler ONLY on the training data


x_train[num_cols_demo] = scaler.fit_transform(x_train[num_cols_demo]) # Write your code here
x_test[num_cols_demo] = scaler.transform(x_test[num_cols_demo]) # Write your code here

x_test
